In [1]:
import numpy as np
import sys, os
sys.path.append('../../shared/utilsGRF/')
from IPython.display import display
import networkx as nx
import subprocess, glob, re
import matplotlib.pyplot as plt

In [2]:
edges=[(1,'a1',2),(2,'a2',3),(3,'a3',4),(4,'a4',1),
       (5,'alpha1',6),(6,'alpha2',7),(7,'alpha3',8),(8,'alpha4',5),
      (1,'kb1-x',5),(5,'ku1',1),(2,'kb2-x',6),(6,'ku2',2),(3,'kb3-x',7),(7,'ku3',3),(4,'kb4-x',8),(8,'ku4',4)]

nnodes=8
MTTfolder='../../shared/utilsGRF' #folder with MTT.py 
basename='graph'

parlist=[x[1] for x in edges]
parlist=list(set(parlist))


for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
print(parlist)

graphname=os.path.join(MTTfolder,basename+'.txt')
outf=open(graphname,'w')
for x in edges:
    x_str=list(map(str,x))
    outf.write('('+','.join(x_str)+') ')
outf.close()

#execute_MTT(self):

args=["python",os.path.join(MTTfolder,'MTT.py'),graphname]
print("executing MTT")
print(subprocess.check_call(args))


#parse_rhos_from_graphfiles(self):
#this was previously called rhos_from_edges
edges=np.array(edges)
#find files that correspond to the spanning trees
files=glob.glob(os.path.join(MTTfolder,basename+'*'))
pat=re.compile('[0-9]+.txt')
n=0
for f in files:
    if pat.findall(f):
        n+=1
all_rhos=[] 
allGraphs=dict()
allterms=dict()
for i in range(1,n+1):
    fname=os.path.join(MTTfolder,basename+'-%d.txt'%i)

    fi=open(fname,'r')
    rho_expr='rho_%d='%i
    allGraphs[i]=[]
    allterms[i]=[]
    for lnum,l in enumerate(fi.readlines()):
        idxs=np.array(list(map(int,l.strip().split())),dtype=bool)
        subset=edges[idxs]
        expr=[x[1] for x in subset] #labels that multiply together in a term of the rho
        term='*'.join(expr)
        if lnum<1:
            rho_expr=rho_expr+term
        else:
            rho_expr=rho_expr+'+'+term
        allterms[i].append(term)
        #now get Graph
        G=nx.DiGraph()
        for edge in subset:
            n0=int(edge[0])
            n1=int(edge[2])
            par=edge[1]
            G.add_edge(n0,n1,name=par)
            G.add_edge
        allGraphs[i].append(G)
        
            
        

    all_rhos.append(rho_expr)

['a4', 'a1', 'alpha2', 'ku2', 'kb3', 'a2', 'kb4', 'alpha1', 'alpha3', 'alpha4', 'kb2', 'a3', 'kb1', 'ku4', 'ku3', 'ku1']
executing MTT
0


In [3]:
import re
kbpat=re.compile("kb")
alphapat=re.compile("alpha[0-9][^r]")
alpharepat=re.compile("alpha[0-9]r")

#Get mapping between trees rooted at bound state and trees rooted at unbound state:
- if there is a binding edge to the root: change for the unbinding edge and that's it
- if there is not a binding edge to the root: reverse minimal path. This should result in less alphas in the tree rooted at the unbound


In [4]:
#the root+term number is an id for the trees

In [5]:

nnodes=4
correspondance=[]
for node in range(nnodes+1,nnodes+1+nnodes):
    
    for n in range(1,nnodes+1): #for trees with n kb edges
        
        #Gs_b_kb=[] #trees with kb directly to root node
        #Gs_b_nokb=[] #trees with no kb directly to root node
        Gs_b=[]
        IDs=[]
        termsb=[]
        for t,term in enumerate(allterms[node]):
            kbs=kbpat.findall(term)
            if len(kbs)==n:
                G=allGraphs[node][t]
                Gs_b.append(G)
                IDs.append(t)
                termsb.append(term)
                #if "kb%d-x"%node in term:
                #    Gs_b_kb.append(G)
                #else:
                #    Gs_b_nokb.append(G)
        
        #now check that for each of those trees there is a corresponding tree rooted at an unbound node
        
        
        for g,G in enumerate(Gs_b):
            
            
            #node_u=node-nnodes
            #if True:
            for node_u in range(1,nnodes+1):
                #look if I have a tree rooted at the unbound that correspond to this graph reversing the minimal path.
                #reversing one binding edge only
                for t,term in enumerate(allterms[node_u]): 
                    kbs=kbpat.findall(term)
                    if len(kbs)==(n-1): #
                    #if True:
                        Gu=allGraphs[node_u][t]
                        mp=nx.shortest_path(Gu,source=node,target=node_u) #since the tree GU is spanning rooted at node_u, for sure there is a path
                        edges_mp=[[mp[i],mp[i+1]] for i in range(len(mp)-1)]

                        #get presumably corresponding tree rooted at unbound from the tree rooted at bound
                        edgesu=[]
                        for edge in G.edges():
                            edge_r=[edge[1],edge[0]]
                            if not edge_r in edges_mp:
                                edgesu.append(",".join(map(str,edge)))
                            else:
                                edgesu.append(",".join(map(str,edge_r)))
                        edgesu=set(edgesu)
                        newedgesu=[",".join(map(str,x)) for x in Gu.edges()]

                        newedgesu=set(newedgesu)
                        if len(edgesu.symmetric_difference(newedgesu))==0:
                            nalphasb=len(alphapat.findall(termsb[g]))
                            nalphasu=len(alphapat.findall(term))
                            if True:
                            #if nalphasb-nalphasu>=0:
                                correspondance.append([node,n,IDs[g],node_u,t,nalphasb-nalphasu])
                            #print("Ge",G.edges())
                            #print("Gue",Gu.edges())
                        
                    
                    
            
                
                        

In [6]:
import pandas as pd

In [7]:
df=pd.DataFrame(np.vstack(correspondance),columns=["node","nkb","IDb","node_u","IDu","alphadif"])
df["unique_u"]=[x[0]+"-"+x[1] for x in list(zip(df["node_u"].astype(str),df["IDu"].astype(str)))]

In [8]:
[len(allterms[x]) for x in allterms]

[40, 40, 40, 40, 40, 40, 40, 40]

In [9]:
ntree=40
for nodeb in range(5,5+nnodes):
    print("nodeb")
    subdf=df[df["node"]==nodeb]
    ar=subdf["IDb"].values
    for i in range(ntree):
        if not i in ar:
            print("not ",i)

nodeb
not  4
not  6
not  10
not  12
not  16
not  18
not  20
not  22
not  25
not  28
not  35
not  37
not  39
nodeb
not  3
not  6
not  9
not  12
not  13
not  16
not  17
not  20
not  21
not  24
not  27
not  30
not  31
nodeb
not  5
not  6
not  11
not  12
not  15
not  19
not  20
not  22
not  23
not  28
not  33
not  36
not  39
nodeb
not  7
not  8
not  11
not  12
not  17
not  18
not  21
not  26
not  27
not  30
not  34
not  37
not  39


In [10]:

for nodeu in range(1,1+nnodes):
    subdf=df[df["node_u"]==node_u]
    ar=subdf["IDu"].values
    for i in range(ntree):
        if not i in ar:
            print("not ",i)

not  0
not  1
not  2
not  3
not  4
not  5
not  6
not  13
not  14
not  15
not  16
not  17
not  18
not  0
not  1
not  2
not  3
not  4
not  5
not  6
not  13
not  14
not  15
not  16
not  17
not  18
not  0
not  1
not  2
not  3
not  4
not  5
not  6
not  13
not  14
not  15
not  16
not  17
not  18
not  0
not  1
not  2
not  3
not  4
not  5
not  6
not  13
not  14
not  15
not  16
not  17
not  18


In [ ]:
#Now compare trees rooted at bound with n kbs and compare to those with n-1 kbs.
#count the alphas in one and the other


In [15]:

nnodes=4
correspondance=[]
for node in range(nnodes+1,nnodes+1+nnodes):
    
    for n in range(1,nnodes+1): #for trees with n kb edges
        
        #Gs_b_kb=[] #trees with kb directly to root node
        #Gs_b_nokb=[] #trees with no kb directly to root node
        Gs_b=[]
        IDs=[]
        termsb=[]
        nalphas=[]
        for t,term in enumerate(allterms[node]):
            kbs=kbpat.findall(term)
            if len(kbs)==n:
                G=allGraphs[node][t]
                Gs_b.append(G)
                IDs.append(t)
                termsb.append(term)
                nalpha=len(alphapat.findall(term))
                nalphas.append(nalpha)
                
                #if "kb%d-x"%node in term:
                #    Gs_b_kb.append(G)
                #else:
                #    Gs_b_nokb.append(G)
        
        nalphasu=[]
        for node_u in range(1,nnodes+1):
                
            for t,term in enumerate(allterms[node_u]): 
                kbs=kbpat.findall(term)
                if len(kbs)==(n-1): #
                    nalpha=len(alphapat.findall(term))
                    nalphasu.append(nalpha)
                    
        print(node,n,max(nalphasu)-max(nalphas))
                        
                    
                    
            
                
                        

5 1 0
5 2 0
5 3 0
5 4 0
6 1 0
6 2 0
6 3 0
6 4 0
7 1 0
7 2 0
7 3 0
7 4 0
8 1 0
8 2 0
8 3 0
8 4 0


#Try to see if I can recover all trees rooted at the unbound nodes doing the following:
1) Trees with an unbinding edge at the root of the unbound cycle:
For each tree rooted at bound: pick a binding edge and reverse it
if this was the root, the bound cycle is ok.
If not, then need to remove the outgoing edge. If this means a subgraph remains disconnected, then add edges to connect it (this means adding alphas).
Regarding the unbound cycle: the new edge becomes root. 
2) Trees with not unbinding edge at the root of the unbound cycle:


OutEdgeView([(2, 3), (3, 4), (4, 1), (5, 6), (6, 2), (7, 8), (8, 5)])